In [33]:
import os
import glob
import time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [89]:
SOURCE_ID = 4006

CORALNET_URL = "https://coralnet.ucsd.edu/"

DATA_ROOT = "C://Users/jordan.pierce/Documents/GitHub/CoralNet_Tools/CoralNet_Data/3420/"

IMAGES = glob.glob(DATA_ROOT + "images/*.*")
LABELS = DATA_ROOT + "annotations.csv"
LABELSET = DATA_ROOT + "label_set.csv"

In [90]:
USERNAME = os.getenv("CORALNET_USERNAME")
PASSWORD = os.getenv("CORALNET_PASSWORD")

In [92]:
def get_status(driver):
    
    # Get the status display to check upload
    status_display = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "status_display"))
    )

    # Get the status details to check upload
    status_detail = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "status_detail"))
    )
        
    return driver, status_display.text, status_detail.text


def login(driver):

    # Navigate to the page to login
    driver.get(CORALNET_URL + "accounts/login/")

    # Find the username and password input fields and the login button
    username_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "id_username"))
    )
    password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "id_password"))
    )
    login_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@type='submit'][@value='Sign in']"))
    )

    # Enter the username and password
    username_input.send_keys(USERNAME)
    password_input.send_keys(PASSWORD)

    # Click the login button
    time.sleep(5)
    login_button.click()

    # Confirm login was successful; after 60 seconds, throw an error.
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@href='/accounts/logout/']/span[text()='Sign out']"))
        )
        print(f"NOTE: Successfully logged in for {USERNAME}")

    except Exception as e:
        raise ValueError(f"ERROR: Could not login with {USERNAME}")
    
    return driver


def upload_images(driver):
    
    # Go to the upload page
    driver.get(CORALNET_URL + f"source/{SOURCE_ID}/upload/images/")

    # Locate the file input field
    file_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@type='file'][@name='files']"))
    )

    # Loop through the list of file paths and send each one to the file input field
    for image_path in IMAGES:
        file_input.send_keys(image_path)

    # Wait up to 30 seconds for each image before trying to upload
    upload_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//button[@id='id_upload_submit']"))
    )

    # Click the upload button once it has appeared
    time.sleep(5)
    upload_button.click()

    try:
        # Confirm upload was successful; after 60 seconds, throw an error.
        WebDriverWait(driver, 30 * len(IMAGES)).until(
            EC.text_to_be_present_in_element((By.ID, "status_display"), "Upload complete")
        )

        print(f"NOTE: Successfully uploaded {len(IMAGES)} images")

    except Exception as e:
        raise ValueError("ERROR: Could not upload images")

    return driver


def upload_labelset(driver):

    # Go to the upload page
    driver.get(CORALNET_URL + f"source/{SOURCE_ID}/labelset/import/")

    # Locate the file input field
    file_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@type='file'][@name='csv_file']"))
    )

    try:
        # Submit the file
        file_input.send_keys(LABELSET)

    except Exception as e:
        raise ValueError("ERROR: Labelset file is not in the correct format")

    # Wait for the button to appear and click it
    save_button = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.ID, "id_upload_submit"))
    )

    # Click the button; wait for the labelsets to be imported
    time.sleep(5)
    save_button.click()

    try:
        # Wait for the status message to appear to know if it was successful
        WebDriverWait(driver, 60).until(
            EC.text_to_be_present_in_element((By.ID, "status_display"), "Labelset saved")
        )
        print(f"NOTE: Successfully uploaded {os.path.basename(LABELSET)}")

    except Exception as e:
        raise ValueError(f"ERROR: Could not upload {os.path.basename(LABELSET)}.")

    return driver
        
    
def upload_labels(driver):

    # Go to the upload page
    driver.get(CORALNET_URL + f"source/{SOURCE_ID}/upload/annotations_csv/")

    try:
        # Locate the file input field; if the labelset wasn't set correctly an error is thrown
        file_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@type='file'][@name='csv_file']"))
        )

    except Exception as e:
        raise ValueError(f"ERROR: Labelset associated with source {SOURCE_ID} has not been set")


    try: 

        # Submit the file
        file_input.send_keys(LABELS)

        while True:

            # Get the status 
            driver, display, details = get_status(driver)

            if "Processing" in display:
                continue

            if "Error" in display:
                raise ValueError(f"ERROR: {display}\n{details}")

            if "Data OK" in display:
                break

        # Wait for the button to appear and click it
        save_button = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.ID, "id_upload_submit"))
        )

        # Click the button; wait for the labelsets to be imported
        time.sleep(5)
        save_button.click()

        while True:

            # Get the status 
            driver, display, details = get_status(driver)

            if "Saving" in display:
                continue

            elif "saved" in display:
                break

            else:
                raise ValueError(f"ERROR: {display}\n{details}")

        print(f"NOTE: Successfully uploaded {os.path.basename(LABELS)}")

    except Exception as e:
        raise ValueError(f"ERROR: Could not upload and/or save {os.path.basename(LABELS)}\n{e}")

    return driver


# Set up the browser driver (in this case, Chrome)
driver = webdriver.Chrome()
driver = login(driver)
driver = upload_images(driver)
driver = upload_labelset(driver)
driver = upload_labels(driver)
driver.close()

NOTE: Successfully logged in for jordan.pierce@noaa.gov
NOTE: Successfully uploaded label_set.csv
NOTE: Successfully uploaded annotations.csv


In [93]:
driver.close()